In [1]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [2]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Conv1D, Flatten, MaxPooling1D, Reshape, Activation, Embedding, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
import pandas as pd

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

/home/zhouz2/.conda/envs/rbenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv('reuters.csv', delimiter='\t', names=['id', 'timestamp', 'title', 'url', 'first_line'])

In [4]:
df = df.set_index('id')

In [5]:
batch_size = 64  # Batch size for training.
epochs = 10000  # Number of epochs to train for.
latent_dim = 1024  # Latent dimensionality of the encoding space (number of nodes per LSTM layer).
num_samples = len(df) #min(2, len(df))  # Number of samples to train on.
num_test_samples = 20
num_conv = 8 # Number of Conv layers
num_dense = num_conv
initial_num_filters = 20 # Initial number of filters; will double after every conv layer
use_dropout = False
dense_size = 1024

In [6]:
input_texts = df['first_line'][:num_samples]
output_texts = df['title'][:num_samples]

In [7]:
input_texts = input_texts.apply(lambda i: i.strip().lower() + ' \n')
output_texts = output_texts.apply(lambda o: '\t '+ o.strip().lower() + ' \n')

In [8]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', oov_token='<unk>')
tokenizer.fit_on_texts(input_texts)
tokenizer.fit_on_texts(output_texts)

In [9]:
input_sequences = tokenizer.texts_to_sequences(input_texts)
output_sequences = tokenizer.texts_to_sequences(output_texts)
word_id_dict = tokenizer.word_index
id_word_dict = dict()
for k in word_id_dict:
    id_word_dict[word_id_dict[k]] = k;

In [10]:
num_samples = len(input_texts)
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in output_sequences])
num_dict_size = len(tokenizer.word_index)
# num_input_tokens = len(input_dict)
# num_output_tokens = len(output_dict)

In [11]:
print('Number of samples:', num_samples)
print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Num of tokens for inputs:', num_input_tokens)
print('Max sequence length for outputs:', max_decoder_seq_length)
# print('Num of tokens for outputs:', num_output_tokens)
print('Number of words in the dictionary (including OOV token):', num_dict_size)

Number of samples: 52302
Max sequence length for inputs: 1107
Max sequence length for outputs: 25
Number of words in the dictionary (including OOV token): 44340


In [12]:
# encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# reversed_encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')
# decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')

In [13]:
# for i in range(num_samples):
#     input_text = input_tokenizer.texts_to_sequences(input_texts[i])
#     input_text = input_dict
#     encoder_input_data[i, :, :] = input_tokenizer.texts_to_sequences(input_texts[i])
#     reversed_encoder_input_data[i, :, :] = encoder_input_data[i, ::-1, :]
#     decoder_input_data[i, :, :] = output_tokenizer.texts_to_sequences(output_texts[i])
#     decoder_target_data[i, :, :] = np.row(decoder_input_data[i, :, :], 1)
#     decoder_target_data[i, 0, :] = 0;

In [14]:
## Baseline 2: CNN
# Define an input sequence and process it.
model = Sequential()
model.add(Reshape((max_encoder_seq_length, 1), input_shape=(max_encoder_seq_length,)))
num_filters = initial_num_filters
for _ in range(num_conv):
    model.add(Conv1D(filters=num_filters, kernel_size=3, activation='relu'))
    num_filters *= 2
    model.add(MaxPooling1D())
    if use_dropout:
        model.add(Dropout(0.5))
        
model.add(Flatten())
for _ in range(num_conv-1):
    model.add(Dense(dense_size, activation='relu'))
model.add(Dense(max_decoder_seq_length, activation='relu'))
model.compile(optimizer='adam', loss='mse')
model.summary()
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)# Compile & run training
# model.compile(optimizer='adam', loss='mse')

# model.summary()
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 1107, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1105, 20)          80        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 552, 20)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 550, 40)           2440      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 275, 40)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 273, 80)           9680      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 136, 80)           0         
__________

In [15]:
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length), dtype='int32')
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length), dtype='int32')

In [16]:
for k in range(num_samples):
    encoder_input_data[k][:len(input_sequences[k])] = input_sequences[k][:]
    decoder_target_data[k][:len(output_sequences[k])-1] = output_sequences[k][1:]

In [ ]:
# encoder_input_data = encoder_input_data.reshape((*encoder_input_data.shape, 1))
# decoder_input_data = decoder_input_data.reshape((*decoder_input_data.shape, 1))
# decoder_target_data = decoder_target_data.reshape((*decoder_target_data.shape, 1))

In [ ]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit(encoder_input_data, decoder_target_data,
          batch_size=batch_size,
          epochs=epochs#,
          #callbacks=[EarlyStopping(monitor='loss', patience=10)]
         )

Epoch 1/10000
52302/52302 [==============================] - 33s 630us/step - loss: 13676110.6950
Epoch 2/10000
52302/52302 [==============================] - 28s 542us/step - loss: 12991241.3586
Epoch 3/10000
52302/52302 [==============================] - 29s 547us/step - loss: 24982849.4500
Epoch 4/10000
52302/52302 [==============================] - 28s 538us/step - loss: 13032691.5608
Epoch 5/10000
52302/52302 [==============================] - 28s 539us/step - loss: 12944545.3266
Epoch 6/10000
52302/52302 [==============================] - 28s 544us/step - loss: 12916646.9797
Epoch 7/10000
52302/52302 [==============================] - 28s 541us/step - loss: 12910394.4325
Epoch 8/10000
52302/52302 [==============================] - 28s 542us/step - loss: 12892217.2356
Epoch 9/10000
52302/52302 [==============================] - 28s 543us/step - loss: 12877508.0019
Epoch 10/10000
52302/52302 [==============================] - 28s 542us/step - loss: 12870312.5076
Epoch 11/10000
5230

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



52302/52302 [==============================] - 28s 532us/step - loss: 12156216.4744
Epoch 51/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12135671.9777
Epoch 52/10000
52302/52302 [==============================] - 28s 534us/step - loss: 12107922.1133
Epoch 53/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12099919.9138
Epoch 54/10000
52302/52302 [==============================] - 28s 532us/step - loss: 12061234.5653
Epoch 55/10000
52302/52302 [==============================] - 28s 534us/step - loss: 12045837.0167
Epoch 56/10000
52302/52302 [==============================] - 28s 533us/step - loss: 127989569.7544
Epoch 57/10000
52302/52302 [==============================] - 28s 534us/step - loss: 12901002.0847
Epoch 58/10000
52302/52302 [==============================] - 28s 532us/step - loss: 12777187.3448
Epoch 59/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12684249.3225
Epoch 60/10000
52302/523

52302/52302 [==============================] - 28s 533us/step - loss: 12869992.7700
Epoch 133/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12729386.3641
Epoch 134/10000
52302/52302 [==============================] - 28s 534us/step - loss: 12551849.1805
Epoch 135/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12285335.1186
Epoch 136/10000
52302/52302 [==============================] - 28s 533us/step - loss: 12085880.1466
Epoch 137/10000
52302/52302 [==============================] - 28s 532us/step - loss: 11727415.4907
Epoch 138/10000
52302/52302 [==============================] - 28s 534us/step - loss: 11575221.1373
Epoch 139/10000
52302/52302 [==============================] - 28s 533us/step - loss: 11525056.5377
Epoch 140/10000
52302/52302 [==============================] - 28s 534us/step - loss: 11401372.2244
Epoch 141/10000
52302/52302 [==============================] - 28s 534us/step - loss: 11310840.8144
Epoch 142/10000


52302/52302 [==============================] - 28s 533us/step - loss: 7304039.3114
Epoch 215/10000
52302/52302 [==============================] - 28s 533us/step - loss: 7305239.1291
Epoch 216/10000
52302/52302 [==============================] - 28s 532us/step - loss: 7319877.1554
Epoch 217/10000
52302/52302 [==============================] - 28s 533us/step - loss: 7184938.2174
Epoch 218/10000
52302/52302 [==============================] - 28s 533us/step - loss: 7103197.9762
Epoch 219/10000
52302/52302 [==============================] - 28s 532us/step - loss: 7016320.4555
Epoch 220/10000
52302/52302 [==============================] - 28s 534us/step - loss: 6995309.9968
Epoch 221/10000
52302/52302 [==============================] - 28s 532us/step - loss: 6994743.0567
Epoch 222/10000
52302/52302 [==============================] - 28s 533us/step - loss: 6949829.7487
Epoch 223/10000
52302/52302 [==============================] - 28s 532us/step - loss: 6852302.5007
Epoch 224/10000
52302/5230

52302/52302 [==============================] - 28s 533us/step - loss: 5169255.3469
Epoch 297/10000
52302/52302 [==============================] - 28s 531us/step - loss: 5065610.6288
Epoch 298/10000
52302/52302 [==============================] - 28s 531us/step - loss: 5057982.8739
Epoch 299/10000
52302/52302 [==============================] - 28s 529us/step - loss: 5161910.6347
Epoch 300/10000
52302/52302 [==============================] - 28s 534us/step - loss: 5087581.5296
Epoch 301/10000
52302/52302 [==============================] - 28s 532us/step - loss: 5014256.3653
Epoch 302/10000
52302/52302 [==============================] - 28s 532us/step - loss: 5028669.2928
Epoch 303/10000
52302/52302 [==============================] - 28s 533us/step - loss: 4999760.4837
Epoch 304/10000
52302/52302 [==============================] - 28s 532us/step - loss: 4977804.1617
Epoch 305/10000
52302/52302 [==============================] - 28s 533us/step - loss: 4975819.9203
Epoch 306/10000
52302/5230

52302/52302 [==============================] - 28s 533us/step - loss: 4312106.7546
Epoch 379/10000
52302/52302 [==============================] - 28s 532us/step - loss: 4228987.2272
Epoch 380/10000
52302/52302 [==============================] - 28s 533us/step - loss: 4188462.4049
Epoch 381/10000
52302/52302 [==============================] - 28s 534us/step - loss: 4210675.9458
Epoch 382/10000
52302/52302 [==============================] - 28s 534us/step - loss: 4224354.1801
Epoch 383/10000
52302/52302 [==============================] - 28s 532us/step - loss: 4188674.0641
Epoch 384/10000
52302/52302 [==============================] - 28s 534us/step - loss: 4193717.4661
Epoch 385/10000
52302/52302 [==============================] - 28s 531us/step - loss: 4122693.2180
Epoch 386/10000
52302/52302 [==============================] - 28s 531us/step - loss: 4115014.4277
Epoch 387/10000
52302/52302 [==============================] - 28s 532us/step - loss: 4109485.2036
Epoch 388/10000
52302/5230

In [ ]:
# # Next: inference mode (sampling).
# # Here's the drill:
# # 1) encode input and retrieve initial decoder state
# # 2) run one step of decoder with this initial state
# # and a "start of sequence" token as target.
# # Output will be the next target token
# # 3) Repeat with the current target token and current states

# # Define sampling models
# encoder_model = Model(encoder_inputs, encoder_states)
# encoder_model.summary()

# decoder_state_input_h = Input(shape=(latent_dim,))
# decoder_state_input_c = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder_mapped_input = decoder_embedding(decoder_inputs)
# decoder_outputs, state_h, state_c = decoder_lstm(decoder_mapped_input, initial_state=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_outputs = Flatten()(decoder_outputs)
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)
# decoder_model.summary()

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = model.predict(input_seq)[0]
    decoded_sentence = ''
    for i in range(len(states_value)):
        int_val = int(round(states_value[i]))
        if int_val > 0:
            decoded_sentence += id_word_dict[int_val] + ' '
            continue
        elif int_val < 0:
            print('negative number encountered')
        
        decoded_sentence += '?' + ' '

    return decoded_sentence

In [ ]:
for seq_index in range(num_test_samples):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Target sentence:', output_texts[seq_index][1:])